In [42]:
import pandas as pd
import numpy as np
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer
nltk.download('vader_lexicon')
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
import spacy

[nltk_data] Downloading package vader_lexicon to C:\Users\Ishaan
[nltk_data]     Sharma\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!
[nltk_data] Downloading package stopwords to C:\Users\Ishaan
[nltk_data]     Sharma\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [45]:
df = pd.read_csv('NewsCategorizer.csv')
df.head()

,category,headline,links,short_description,keywords
0,WELLNESS,143 Miles in 35 Days: Lessons Learned,https://www.huffingtonpost.com/entry/running-l...,Resting is part of training. I've confirmed wh...,running-lessons
1,WELLNESS,Talking to Yourself: Crazy or Crazy Helpful?,https://www.huffingtonpost.com/entry/talking-t...,Think of talking to yourself as a tool to coac...,talking-to-yourself-crazy
2,WELLNESS,Crenezumab: Trial Will Gauge Whether Alzheimer...,https://www.huffingtonpost.com/entry/crenezuma...,The clock is ticking for the United States to ...,crenezumab-alzheimers-disease-drug
3,WELLNESS,"Oh, What a Difference She Made",https://www.huffingtonpost.com/entry/meaningfu...,"If you want to be busy, keep trying to be perf...",meaningful-life
4,WELLNESS,Green Superfoods,https://www.huffingtonpost.com/entry/green-sup...,"First, the bad news: Soda bread, corned beef a...",green-superfoods


In [46]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 5 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   category           50000 non-null  object
 1   headline           50000 non-null  object
 2   links              50000 non-null  object
 3   short_description  50000 non-null  object
 4   keywords           47332 non-null  object
dtypes: object(5)
memory usage: 1.9+ MB


In [47]:
df.isnull().sum()

category                0
headline                0
links                   0
short_description       0
keywords             2668
dtype: int64

In [48]:
df['keywords'].unique()

array(['running-lessons', 'talking-to-yourself-crazy',
       'crenezumab-alzheimers-disease-drug', ...,
       'san-jose-giants-japanese-heritage-night',
       'dab-kid-san-antonio-spurs', 'the-first-simone-biles'],
      dtype=object)

In [49]:
df.dropna(subset=['keywords'], inplace=True)

In [50]:
#dropping duplicates
print('the number of articles before processing :',len(df))
df.drop_duplicates(subset=['headline','short_description'],inplace=True)
print('The number of articles after processing :',len(df))

the number of articles before processing : 47332
The number of articles after processing : 43370


In [51]:
#Getting Headlines with more than 4 words
print('the number of articles before processing :',len(df))
df=df[df['headline'].apply((lambda x: len(x.split())>=4))]
print('The number of articles after processing :',len(df))

the number of articles before processing : 43370
The number of articles after processing : 42098


In [52]:
df = df.rename(columns= {'short_description' : 'abstract'}, inplace=False)
df['text'] = df['headline'] + " " + df['abstract']

In [53]:
df.head()

,category,headline,links,abstract,keywords,text
0,WELLNESS,143 Miles in 35 Days: Lessons Learned,https://www.huffingtonpost.com/entry/running-l...,Resting is part of training. I've confirmed wh...,running-lessons,143 Miles in 35 Days: Lessons Learned Resting ...
1,WELLNESS,Talking to Yourself: Crazy or Crazy Helpful?,https://www.huffingtonpost.com/entry/talking-t...,Think of talking to yourself as a tool to coac...,talking-to-yourself-crazy,Talking to Yourself: Crazy or Crazy Helpful? T...
2,WELLNESS,Crenezumab: Trial Will Gauge Whether Alzheimer...,https://www.huffingtonpost.com/entry/crenezuma...,The clock is ticking for the United States to ...,crenezumab-alzheimers-disease-drug,Crenezumab: Trial Will Gauge Whether Alzheimer...
3,WELLNESS,"Oh, What a Difference She Made",https://www.huffingtonpost.com/entry/meaningfu...,"If you want to be busy, keep trying to be perf...",meaningful-life,"Oh, What a Difference She Made If you want to ..."
5,WELLNESS,Bad Love Advice: 9 Lessons To Unlearn,https://www.huffingtonpost.com/entry/bad-love-...,"By Carey Moss for YouBeauty.com Love rom-coms,...",bad-love-advice-from-movies,Bad Love Advice: 9 Lessons To Unlearn By Carey...


In [54]:
#Stopword Removal
stopword = set(stopwords.words('english'))
def clean_stopwords(text):
    text = ' '.join(word for word in text.split() if word not in stopword)
    return text

df['text'] = df['text'].apply(clean_stopwords)

In [55]:
#Stemming
ps = PorterStemmer()
def porterstemmer(text):
  text = ' '.join([ps.stem(word) for word in text.split() if word in text])
  return text
df['text'] = df['text'].apply(porterstemmer)

In [56]:
#Lemmatization
nlp = spacy.load('en_core_web_sm')
def lemmatization (text):
    doc = nlp(text)
    tokens = [token.lemma_.lower() for token in doc if not token.is_stop and not token.is_punct]
    return ' '.join(tokens)
df['text'] = df['text'].apply(lemmatization)

In [57]:
df.isnull().sum()

category    0
headline    0
links       0
abstract    0
keywords    0
text        0
dtype: int64

In [58]:
df.head()

,category,headline,links,abstract,keywords,text
0,WELLNESS,143 Miles in 35 Days: Lessons Learned,https://www.huffingtonpost.com/entry/running-l...,Resting is part of training. I've confirmed wh...,running-lessons,143 mile 35 day lesson learn rest training i'v...
1,WELLNESS,Talking to Yourself: Crazy or Crazy Helpful?,https://www.huffingtonpost.com/entry/talking-t...,Think of talking to yourself as a tool to coac...,talking-to-yourself-crazy,talk crazi crazi helpful think talk tool coach...
2,WELLNESS,Crenezumab: Trial Will Gauge Whether Alzheimer...,https://www.huffingtonpost.com/entry/crenezuma...,The clock is ticking for the United States to ...,crenezumab-alzheimers-disease-drug,crenezumab trial gaug alzheimer drug prevent s...
3,WELLNESS,"Oh, What a Difference She Made",https://www.huffingtonpost.com/entry/meaningfu...,"If you want to be busy, keep trying to be perf...",meaningful-life,oh differ want busy tri perfect want happy foc...
5,WELLNESS,Bad Love Advice: 9 Lessons To Unlearn,https://www.huffingtonpost.com/entry/bad-love-...,"By Carey Moss for YouBeauty.com Love rom-coms,...",bad-love-advice-from-movies,bad love advice 9 lesson unlearn carey moss yo...
